In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification
from datasets import load_from_disk
from sklearn.metrics import accuracy_score, f1_score

train_dataset = load_from_disk("train_dataset")
val_dataset = load_from_disk("val_dataset")

print("Train dataset size:", len(train_dataset))
print("Val dataset size:", len(val_dataset))

# Try loading first row
print("Sample:", train_dataset[0])


batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2
)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

epochs = 3
for epoch in range(epochs):
    model.train()
    total_train_loss = 0

    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

    print(f"Epoch {epoch+1} Loss = {total_train_loss/len(train_loader)}")

# Save model
model.save_pretrained(r"C:\Users\masoka\Documents\ai-jailbreak-detector\training")
print("Model saved to C:\\Users\\masoka\\Documents\\ai-jailbreak-detector\\training")